In [95]:
using JuMP, Gurobi, Random, CSV, DataFrames, Statistics, JLD2, FileIO, DelimitedFiles

model_type="model_without_k";
include("scripts/"*model_type*"/display.jl"); include("scripts/"*model_type*"/model.jl"); include("scripts/"*model_type*"/run.jl"); include("data/map_config.jl");

In [85]:
# Benchmarks definition
b1=Dict("Name"=> "1)", "Transfer"=> false, "Flexible" => false, "High capacity" => false);
b2=Dict("Name"=> "2)", "Transfer"=> false, "Flexible" => false, "High capacity" => true); # We take high capacity busses
b3=Dict("Name"=> "3)", "Transfer"=> false, "Flexible" => true, "High capacity" => true); # We take high capacity busses + Flexible assign
b4=Dict("Name"=> "4)", "Transfer"=> true, "Flexible" => true, "High capacity" => true); # We take high capacity busses + Flexible assign + Transfer

### Inputs

In [86]:
# Map Inputs
map_type="clust_map_5";

# Model Inputs
benchmark=b4;
G = 0.5 # max detour ratio for customers (traveling time < (1+G)gamma)
Gtype= "shortestpathpercent" # or "absolutetime"

map_inputs,Wk,Q=create_inputs(map_type,benchmark);
(map_title,hubs_id,vbs_id,nb_locs,cust_id,nb_cust,depot_locs,horizon,tstep,park_slots)=map_inputs;
model_inputs = (G,Gtype,Wk,Q);

### Display map & create network

In [87]:
data, map1, tsnetwork, params, abbrev, pre_time = create_network(map_inputs, model_inputs,benchmark);
q, t, I, K=abbrev;

In [88]:
# Show map
#display(map1)
#print_cust_bus_details(params,abbrev,data.locs_id)

In [83]:
# Print number of nodes and arcs
#print_nb_arcs(tsnetwork,params,I)


In [84]:
# To visualize some arcs
#timespaceviz_arcs("viz.png", horizon, tstep, tsnetwork, tsnetwork.A_plus[56],data.locs_id,data.locs_desc, x_size=2000, y_size=1000)

### Run model & Save results

In [89]:
# Model objective coefficients
mu=1 # coefficient in front of walking time
beta=1 # coefficient in front of waiting time
lambda=1 # transfer penalty
alpha1=1 # coefficient in front of vehicle driving time 
nu=20 # coefficient in front of vehicle utilization
alpha2=0 # coefficient in front of unmet demand
coefficients=mu,beta,lambda,alpha1,nu,alpha2;

In [91]:
sol=network_model(Q,abbrev,data.wo,tsnetwork,params,coefficients,1);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 252446 rows, 71143 columns and 446635 nonzeros
Model fingerprint: 0xb6343e2e
Variable types: 0 continuous, 71143 integer (71143 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+01, 7e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 216389 rows and 42146 columns
Presolve time: 4.56s
Presolved: 36057 rows, 28997 columns, 152844 nonzeros
Variable types: 0 continuous, 28997 integer (28997 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4876    8.2595477e+04   7.193843e-01   1.855998e+09      5s
Concurrent spin time: 0.01s


In [92]:
# Fix z and resolve
sol2=network_model_zfixed(Q,abbrev,data.wo,tsnetwork,params,coefficients,sol.z,1);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 252446 rows, 64188 columns and 357495 nonzeros
Model fingerprint: 0xd8763da5
Variable types: 0 continuous, 64188 integer (64188 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [4e+02, 7e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 252446 rows and 64188 columns
Presolve time: 0.08s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.11 seconds (0.22 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 53463.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.346354870463e+04, best bound 5.346354870463e+04, gap 0.0000%

User-callback calls 713, time in user-callback 0.00 sec
Objective val

In [94]:
# Fix z and xi and resolve
sol3=network_model_z_xifixed(Q,abbrev,data.wo,tsnetwork,params,coefficients,sol.z,sol.xi,1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 252446 rows, 63915 columns and 293125 nonzeros
Model fingerprint: 0x4bd6d82f
Variable types: 0 continuous, 63915 integer (63915 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e+02, 7e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 53463.548705
Presolve removed 252446 rows and 63915 columns
Presolve time: 0.04s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.11 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 53463.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.346354870463e+04, best bound 5.346354870463e+04, gap 0.0000%

User-callback calls 

(xi =   [1, Dict{String, Real}("transfer" => 0, "wo" => 1292.5, "walking" => 1788.14, "o" => 4, "wd" => 495.65, "d" => 6)                 ]  =  1.0
  [10, Dict{String, Real}("transfer" => 0, "wo" => 1277.25, "walking" => 1986.17, "o" => 4, "wd" => 708.91, "d" => 5)               ]  =  0.0
  [2, Dict{String, Any}("transfer" => 1, "wo" => 1338.54, "walking" => 2010.54, "o" => 4, "h" => [2, 11], "wd" => 672.0, "d" => 6)  ]  =  0.0
  [2, Dict{String, Any}("transfer" => 1, "wo" => 1338.54, "walking" => 2010.54, "o" => 4, "h" => [4, 12], "wd" => 672.0, "d" => 6)  ]  =  0.0
  [2, Dict{String, Any}("transfer" => 1, "wo" => 528.85, "walking" => 1200.85, "o" => 2, "h" => [4, 12], "wd" => 672.0, "d" => 6)   ]  =  0.0
  [2, Dict{String, Real}("transfer" => 0, "wo" => 528.85, "walking" => 1200.85, "o" => 2, "wd" => 672.0, "d" => 6)                  ]  =  -0.0
  [3, Dict{String, Any}("transfer" => 1, "wo" => 432.0, "walking" => 1855.2, "o" => 3, "h" => [2, 11], "wd" => 1423.2, "d" => 7)    ]  =  0.0

Save solution

In [87]:
save_res = true; # to save the results in the result folder
include("scripts/"*model_type*"/display.jl")
if save_res
    benchmark_name=string(benchmark["Name"])*"Tr_"*string(benchmark["Transfer"])*"_Flex_"*string(benchmark["Flexible"])*"_HighCap_"*string(benchmark["High capacity"])
    resultfolder="results/"*model_type*"/"*map_type*"/"*benchmark_name*"/";
    #resultfolder="results/"*model_type*"/"*map_type*"/";
    
    if !isdir(resultfolder)
        mkdir(resultfolder)
    end
    jldsave(resultfolder*"solution.jld2", xi=sol.xi, x=sol.x, z=sol.z)
    write_result(resultfolder*"res.txt",sol,tsnetwork,params,abbrev,data.wo,data.locs_id,false)
end

### Visualizations

In [88]:
#xi,x,z=load(resultfolder*"solution.jld2", "xi", "x", "z");

In [81]:
#timespaceviz_arcs(resultfolder*"arcsplot.png",horizon, tstep, tsnetwork, params.Ai[1], data.locs_id,data.locs_desc; x_size=2000, y_size=1000)

In [82]:
#timespaceviz_bus(resultfolder*"tsplot.png", horizon, tstep, tsnetwork, params, sol.z, data.locs_id, data.locs_desc,x_size=2000, y_size=1000)

In [83]:
#ts=0 # time from which we want to print the arcs
#map2=print_traveling_arcs(sol,ts,map1,data.locs,tsnetwork,params,horizon,data.locs_id,data.locs_desc,true,save_res,resultfolder)